# CNNs From Scratch
In class, we made versions of LeNet and a custom model called LeNet2 where we duplicated all the convolutional layers. We'll try those, in addition to building a new network and trying to train an existing network without its pretrained weights.

## Imports

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.utils import to_categorical
from keras.applications import EfficientNetB0
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.preprocessing import MinMaxScaler
import os

## Load the data

In [ ]:
train_data = np.load('Noise_Train.npz')
test_data = np.load('Noise_Test.npz')

In [ ]:
X_train = train_data['images']
y_train = train_data['labels']

X_test = test_data['images']
y_test = test_data['labels']

label_names = train_data['label_names']

## Standardize the pixel data

In [ ]:
m_train = X_train.shape[0]
X_train = X_train.reshape(m_train, 32, 32, 1)

m_test = X_test.shape[0]
X_test = X_test.reshape(m_test, 32, 32, 1)

X_train.shape

(100000, 32, 32, 1)

## LeNet and LeNet2

In [ ]:
def build_LeNet(input_shape=(32, 32, 1), outputs=2):
    X0 = Input(shape=input_shape)
    X1 = Conv2D(20, kernel_size=(5, 5), padding='same', input_shape=input_shape, activation='relu')(X0)
    X1_pool = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X1)
    X2 = Conv2D(50, kernel_size=(5, 5), padding='same', activation='relu')(X1_pool)
    X2_pool = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(X2)
    X2_flat = Flatten()(X2_pool)
    X3 = Dense(500, activation='relu')(X2_flat)
    Ph = Dense(outputs, activation='softmax')(X3)
    return Model(inputs=[X0], outputs=[Ph])
    # return Model(inputs=[X0], outputs=[X1, X1_pool, X2, X2_pool, X2_flat, X3, Ph])

In [ ]:
LeNet = build_LeNet()
LeNet.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 32, 32, 20)        520       
                                                                 
 max_pooling2d (MaxPooling2  (None, 16, 16, 20)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 16, 16, 50)        25050     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 8, 8, 50)          0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 3200)              0     

### We'll define some hyperparameters to use for all of our models

In [ ]:
# callbacks
checkpt = ModelCheckpoint('LeNet_Noise_Filter.h5', save_best_only=True, verbose=0)
tb = TensorBoard(log_dir='tb_logs')
e_stop = EarlyStopping(patience=3)

In [ ]:
batch_size = 100
epochs = 10
learning_rate = 2e-4
optimizer = Adam(learning_rate=learning_rate)
loss = 'sparse_categorical_crossentropy'

In [ ]:
LeNet.compile(loss=loss, optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])

In [ ]:
lenet_hist = LeNet.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2, callbacks=[checkpt, tb, e_stop])

Epoch 1/10
800/800 [==============================] - 16s 7ms/step - loss: 0.8723 - accuracy: 0.7403 - val_loss: 0.4408 - val_accuracy: 0.7915
Epoch 2/10
 22/800 [..............................] - ETA: 3s - loss: 0.4237 - accuracy: 0.8041

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


800/800 [==============================] - 4s 5ms/step - loss: 0.4220 - accuracy: 0.8063 - val_loss: 0.4037 - val_accuracy: 0.8159
Epoch 3/10
800/800 [==============================] - 5s 6ms/step - loss: 0.3818 - accuracy: 0.8288 - val_loss: 0.3839 - val_accuracy: 0.8313
Epoch 4/10
800/800 [==============================] - 5s 6ms/step - loss: 0.3410 - accuracy: 0.8495 - val_loss: 0.3602 - val_accuracy: 0.8429
Epoch 5/10
800/800 [==============================] - 4s 6ms/step - loss: 0.3120 - accuracy: 0.8651 - val_loss: 0.3747 - val_accuracy: 0.8343
Epoch 6/10
800/800 [==============================] - 4s 5ms/step - loss: 0.2808 - accuracy: 0.8810 - val_loss: 0.3369 - val_accuracy: 0.8564
Epoch 7/10
800/800 [==============================] - 6s 7ms/step - loss: 0.2491 - accuracy: 0.8955 - val_loss: 0.3467 - val_accuracy: 0.8540
Epoch 8/10
800/800 [==============================] - 5s 6ms/step - loss: 0.2205 - accuracy: 0.9081 - val_loss: 0.3447 - val_accuracy: 0.8601
Epoch 9/10
800/80

In [ ]:
test_loss, test_accuracy = LeNet.evaluate(X_test, y_test)

625/625 [==============================] - 3s 4ms/step - loss: 0.3822 - accuracy: 0.8547
